In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
train_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(3),
                            transforms.ToTensor()
                                      ])
train_data = datasets.ImageFolder(root = 'img',transform=train_transforms)
RATIO = 0.8
n_train_examples = int(len(train_data) * RATIO)
n_orginalvalid_examples = len(train_data) - n_train_examples
n_valid_examples = int((n_orginalvalid_examples)*0.5)
n_test_examples = n_orginalvalid_examples - n_valid_examples
print(n_train_examples)
print(n_valid_examples)
print(n_test_examples)

train_data, valid_data, test_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples, n_test_examples])

640
80
80


In [4]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        
        self.fc_1 = nn.Linear(9, 6)
        self.fc_2 = nn.Linear(6, 3)
        self.fc_3 = nn.Linear(3, output_dim)

    def forward(self, x):

        
        
        x = x.view(x.shape[0], -1)
        h = x
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        x = F.relu(x)
        x = self.fc_3(x)

        
        
        return x, h


In [5]:
OUTPUT_DIM = 2

model = LeNet(OUTPUT_DIM)

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 89 trainable parameters


In [7]:
optimizer = optim.Adam(model.parameters())

In [8]:
BATCH_SIZE=8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [9]:
optimizer = optim.Adam(model.parameters())

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
device

device(type='cuda')

In [13]:
model = model.to(device)
criterion = criterion.to(device)

In [14]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [15]:
from tqdm import tqdm

In [16]:

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs





In [17]:
plot_val_acc=[]
plot_train_acc=[]

plot_train_loss=[]
plot_val_loss=[]
epoch_x_axis=[i for i in range(0,100)]

In [18]:
EPOCHS = 30

best_valid_loss = float('inf')

for epoch in tqdm(range(EPOCHS)):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    plot_val_acc.append(train_acc)
    plot_train_acc.append(valid_acc)

    plot_train_loss.append(train_loss)
    plot_val_loss.append(valid_loss)


  3%|██▊                                                                                | 1/30 [00:02<01:00,  2.07s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.614 | Train Acc: 68.91%
	 Val. Loss: 0.591 |  Val. Acc: 65.00%


  7%|█████▌                                                                             | 2/30 [00:03<00:51,  1.83s/it]

Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.511 | Train Acc: 68.91%
	 Val. Loss: 0.463 |  Val. Acc: 65.00%


 10%|████████▎                                                                          | 3/30 [00:04<00:44,  1.66s/it]

...

Epoch: 28 | Epoch Time: 0m 1s
	Train Loss: 0.031 | Train Acc: 100.00%
	 Val. Loss: 0.034 |  Val. Acc: 100.00%


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [00:36<00:01,  1.24s/it]

Epoch: 29 | Epoch Time: 0m 1s
	Train Loss: 0.030 | Train Acc: 100.00%
	 Val. Loss: 0.032 |  Val. Acc: 100.00%


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:37<00:00,  1.26s/it]

Epoch: 30 | Epoch Time: 0m 1s
	Train Loss: 0.028 | Train Acc: 100.00%
	 Val. Loss: 0.030 |  Val. Acc: 100.00%


In [19]:
test_transforms = transforms.Compose([
                            transforms.Grayscale(num_output_channels=1),
                            transforms.Resize(32),
                            transforms.ToTensor()
                                      ])

test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE)
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.033 | Test Acc: 100.00%
